In [1]:
import pandas as pd
import json
import numpy as np
from google.cloud import bigquery

In [2]:
#conversion du fichier json en json line delimiter pour utiliser pd.read_json avec une chunk size. (Nombre de ligne processée en une fois)
#!cat ../raw_data/export_23-5_minified.json | jq -c '.[]' > ../raw_data/export_converted_2.json

In [3]:
#Nombre de lignes totales dans le JSON : 1320229

In [4]:
#Localisation du projet name et table_id sur gcp
project="marbotic"
table_id = "marbotic_dataset.user_properties"

In [5]:
#intégration des credentials 
from google.oauth2 import service_account

key_path = "/Users/antonin/code/AntoninAnq/gcp/marbotic-7d02fac30bd8.json"

credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

In [6]:
# Observation de la colonne Games, on a des listes de jeu. Sur le chunk de 100K on a 158 combinaisons différentes
tmp = pd.read_csv('../raw_data/user_properties.csv')
#tmp
#pd.DataFrame(tmp.Games.to_list())
#.event_id.value_counts()
#.Games.value_counts().head(20)

/var/folders/c7/kdqvz76x679f28lhd235r_v40000gn/T/ipykernel_87728/2837118331.py:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  tmp = pd.read_csv('../raw_data/user_properties.csv')


In [7]:
job_config = bigquery.LoadJobConfig(
    schema = [
    bigquery.SchemaField("Newsletter", "STRING"),
    #bigquery.SchemaField("Products", "STRING"),
    #bigquery.SchemaField("Pieces", "STRING"),
    bigquery.SchemaField("Type", "STRING"),
    bigquery.SchemaField("Country", "STRING"),
    bigquery.SchemaField("Language", "STRING"),
    bigquery.SchemaField("ProfileType2", "STRING"),
    bigquery.SchemaField("ReceiveNewsletter", "STRING"),
    bigquery.SchemaField("LoggedIn", "STRING"),
    bigquery.SchemaField("event_id", "INT64"),
    bigquery.SchemaField(
        "Games",
        "RECORD",
        mode="REPEATED",
        fields=[
            bigquery.SchemaField("Marbotic", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("Smart Shapes", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("Alphamonster", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("10 Fingers", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("Up To 100", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("Lil Reader", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("More Or Less", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("Vocabubble", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("Marbotic Numbers", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("Bla Bla Box", "STRING", mode="NULLABLE"),
            bigquery.SchemaField("Marbotic Letters", "STRING", mode="NULLABLE"),
        ],
    ),
])

In [11]:
CHUNKSIZE = 100000
table_id = "marbotic_dataset.user_properties"

file_path = '../raw_data/export_converted_2.json'
df = pd.read_json(file_path, lines=True, chunksize=CHUNKSIZE) #Chunk size pour avoir 14 chunk

client = bigquery.Client(project,credentials)

for i,c in enumerate(df):
    index_init = np.linspace(1,len(c),len(c),dtype='int32')
    ep=pd.DataFrame()
    c.reset_index(inplace=True)
    ep=pd.DataFrame(c["user_properties"].to_list()).copy()

    for col_concat in ['Products', 'Pieces', 'Games']:
        cat = ep[f'{col_concat}'].copy().map(lambda x: ", ".join(x)
                                            if isinstance(x, list) else x)
        names = list(
            set([
                x.strip().strip("'").strip("['")
                for x in ', '.join(', '.join(', '.join(
                    list(set([str(x) for x in cat]))).split('\n')).split(
                        "' '")).split(',')
            ]))
        names_transf = [
            col_concat + '_' + x.replace(' ', '_') for x in names
            if x != 'nan' and x != ''
        ]
        names = [x for x in names if x != 'nan' and x != '']
        for ind, name in enumerate(names_transf):

            ep[f'{name}'] = ep[f'{col_concat}'].map(
                lambda x: 1 if isinstance(x, str) and f'{names[ind]}' in x else
                (1 if isinstance(x, list) and
                 len([n for n in x if f'{names[ind]}' in n]) > 0
                 else 0))
            #ep['test']=ep[f'{col_concat}'].str.replace('[^\w\s\d]', '', regex=True)

    ep.drop(['Products', 'Pieces', 'Games'], axis=1, inplace=True)
    ep['event_id']=c["event_id"].copy()
    ep['client_event_time']=c['client_event_time'].copy()
    ep['user_creation_time']=c['user_creation_time'].copy()
    ep['user_id']=c['user_id'].copy()
    ep['user_id']=ep['user_id'].astype(str)

    ep['id'] = (i*CHUNKSIZE) + index_init
    job = client.load_table_from_dataframe(ep, table_id)
    job.result()  # Wait for the job to complete.
    table = client.get_table(table_id)  # Make an API request.
    print(
        "Loaded {} rows and {} columns to {}".format(
            table.num_rows, len(table.schema), table_id
        )
        )

Loaded 500000 rows and 40 columns to marbotic_dataset.user_properties
Loaded 600000 rows and 40 columns to marbotic_dataset.user_properties
Loaded 700000 rows and 40 columns to marbotic_dataset.user_properties
Loaded 800000 rows and 40 columns to marbotic_dataset.user_properties


ArrowTypeError: Expected a string or bytes dtype, got int64

In [ ]:
#colonne à ajouter à la main : Products_Sensory_kit en entier